In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
from API_Tokens import iex_token as token

In [2]:
stocks = pd.read_csv('../sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [3]:
def chunks(lst, x):
    '''
    Returns a list of lists each with the length of the specified batch_size in order to pass into a batch call for an API
    '''
    for i in range(0,len(lst),x):
        yield lst[i:i+x]
        
def get_batches(column, batch_size):
    '''
    Creates a list of lists of tickers
    params:
    column: the tickers you'd like to batch together
    batch_size: how many tickers per batch (AlphaVantage has batch calls up to 100)
    '''
    batches = list(chunks(column, batch_size))
    strings = []
    for i in range(0,len(batches)):
        strings.append(','.join(batches[i]))
    return strings

def create_df(symbol_strings, token):
    '''
    Creates a pandas Dataframe that contains stock price & market cap for specified tickers, as well as the number of 
    shares to buy in relation to the specified portfolio size. This is an equal weigth screener so this calculation evenly
    distributes weight.
    '''
    df = pd.DataFrame(columns=['Ticker','Stock Price','Market Capitalization','# of shares to buy'])
    for i in symbol_strings:
        batch_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={i}&token={token}'
        data = requests.get(batch_call_url).json()
        for symbol in i.split(','):
            df = df.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],index=df.columns),
            ignore_index=True
            )
    return df

def get_number_of_shares(df, portfolio_size):
    '''
    Returns a dataframe with the Number of Shares column populated for each ticker
    parameters:
    df: the dataframe you'd like to calculate number of shares for
    portfolio_size: Amount invested in USD taken as input
    '''
    try:
        val = float(portfolio_size)
    except ValueError:
        print('Please enter value as a number only')
        portfolio_size = input('Enter the value of your portfolio:')
        val = float(portfolio_size)
    position_size = val/len(df.index)
    for i in range(0,len(df.index)):
        df.loc[i,'# of shares to buy'] = math.floor(position_size/df.loc[i,'Stock Price'])
    return df

In [4]:
def write_to_xlsx(df,file_name):
    '''
    Creates a writer object that writes the dataframe to an Excel workbook with some predefined formatting
    params:
    df: Dataframe that is being written to an Excel workbook
    file_name: The name for the file that will be written
    **the file will be placed in the same folder or working directory as this notebook
    '''
    writer = pd.ExcelWriter(f'{file_name}.xlsx',engine='xlsxwriter')
    df.to_excel(writer,file_name,index=False)
    
    background_color = '#4acded'
    font_color = '#000000'
    align = 'center'
    header_frmt = writer.book.add_format({
        'font_color':font_color,
        'bg_color': '9b9c9c',
        'bold':'True',
        'align':align
    })
    str_frmt = writer.book.add_format({
        'font_color':font_color,
        'bg_color':background_color,
        'border':1,
        'align':align
    })
    dollar_frmt = writer.book.add_format({
        'num_format':'$0.00',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1,
        'align':align
    })
    int_frmt = writer.book.add_format({
        'num_format':'0',
        'font_color':font_color,
        'bg_color':'#9ce148',
        'border':1,
        'align':align
    })
    
    column_formats = { 
                    'A': ['Ticker', str_frmt],
                    'B': ['Stock Price', dollar_frmt],
                    'C': ['Market Capitalization', dollar_frmt],
                    'D': ['# of shares to buy', int_frmt]
                    }

    for column in column_formats.keys():
        writer.sheets[file_name].set_column(f'{column}:{column}', 20, column_formats[column][1])
        writer.sheets[file_name].write(f'{column}1', column_formats[column][0], header_frmt)
    writer.save()

In [6]:
def main(tickers, batch_size, portfolio_size,export = False):
    symbol_strings = get_batches(tickers,batch_size)
    df = create_df(symbol_strings, token)
    df = get_number_of_shares(df, portfolio_size)
    
    if export:
        file_name = input('Enter the name for the xlsx file: ')
        write_to_xlsx(df, file_name)
    return df

In [7]:
batch_size = int(input('Enter the batch size: '))
portfolio_size = input('Enter the value of your portfolio:')

Enter the batch size: 100
Enter the value of your portfolio:10000000


In [8]:
df = main(stocks['Ticker'],batch_size,portfolio_size,export=True)

Enter the name for the xlsx file: Recommended_trades


In [9]:
df

,Ticker,Stock Price,Market Capitalization,# of shares to buy
0,A,172.49,52315782321,114
1,AAL,19.00,12019344944,1042
2,AAP,214.72,14187280300,92
3,AAPL,153.34,2523284969320,129
4,ABBV,122.96,212369727169,161
...,...,...,...,...
500,YUM,140.64,40441449015,140
501,ZBH,151.27,31394035008,130
502,ZBRA,581.23,31956759606,34
503,ZION,56.27,8874599476,351
